In [1]:
import Quandl
import numpy as np
import pandas as pd
import math
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import os

%matplotlib inline  
pd.options.display.mpl_style = 'default'
pylab.rcParams['figure.figsize'] = 12, 6 

In [2]:
#ge = get(["GE"])
#ibm = get('IBM')
#msft = get('MSFT')
#ebay = get("EBAY")
#swks = get("SWKS")
#cvx = get("CVX")
#mu = get('MU')
#gild = get("GILD")
#aapl = get('AAPL')

In [4]:
get(['aapl', 'ge'], 'log_returns')

,aapl,ge
Date,,
2001-01-02,0.096074,0.088795
2001-01-03,0.041127,0.005215
2001-01-04,-0.041127,-0.015728
2001-01-05,0.011385,-0.037690
2001-01-08,0.037041,-0.020791
2001-01-09,-0.037041,0.001400
2001-01-10,0.083231,0.041102
2001-01-11,-0.046189,-0.018971
2001-01-12,-0.003643,0.036270


In [3]:
def get(ticker_list, kind = 'log_returns'):
    #takes in a ticker list and returns either a dataframe of closing price 'returns'
    #for each stock or closing 'prices'
    final_df = pd.DataFrame()
    for item in ticker_list:
        ticker = item
        temp_df = Quandl.get('YAHOO/'+str(ticker), 
                                         trim_start='2001-01-01', authtoken="H4uWGQ3jQdm96V7H2Abm")
        temp_df.index = pd.to_datetime(temp_df.index)
        temp_df = temp_df.sort()
        if (kind == 'prices'):
            add_series = temp_df.Close
        elif (kind == 'log_prices'):
            add_series = np.log(temp_df.Close);
        elif (kind == 'returns'):
            add_series = temp_df.Close.shift(-1) - temp_df.Close
        else:
            add_series = np.log(temp_df.Close).shift(-1) - np.log(temp_df.Close)
        final_df[ticker] = add_series
        

    final_df = final_df.sort()
    return final_df


In [5]:
portfolio = pd.DataFrame()
portfolio=portfolio.append(aapl.Close)
portfolio=portfolio.append(ge.Close)
portfolio=portfolio.append(mu.Close)
portfolio=portfolio.append(aapl.Close)


portfolio = portfolio.transpose()
portfolio.columns = ['aapl', 'ge', 'none', 'mu']
portfolio = portfolio[['aapl', 'ge', 'mu']]
portfolio = portfolio.diff()
#portfolio

NameError: name 'aapl' is not defined

In [ ]:
risk = portfolio_risk(portfolio, [.3, .3, .4])
return1 = np.matrix(portfolio.mean())
return1 * np.matrix([.3, .3, .4]).transpose()
sharpe = return1 * np.matrix([.3, .3, .4]).transpose() / risk
sharpe * 240


In [6]:
def portfolio_risk(price_df, weight_list):
    #Take in a PRICE_DF with columns being each stock's daily log_returns
    #Take in WEIGHT_LIST with each list entry corresponding to the respective price_df
    #column's weight in the portfolio
    #PRICE_DF should be a concatenation of return series from return_series()
    if (sum(weight_list) != 1):
        print("EROR: Weights do not sum to 1");
        return None
    cov_matrix = np.matrix(price_df.cov())
    weight_t = np.matrix(weight_list)
    weight = weight_t.transpose()
    variance = weight_t * cov_matrix * weight
    return variance
    

In [13]:
portfolio_risk(returns[['ge', 'msft', 'aapl', 'mu']], [.2, .2, .4, .2])



NameError: name 'returns' is not defined

In [11]:
def return_series(price_df, col):
    #Take in a price_df for one security
    #Return a series of the daily log returns of the price_df[col] column
    price_df.index = pd.to_datetime(price_df.Date)
    price_df = price_df.sort()
    rets = np.log(price_df[col]).diff()
    return (rets)


In [15]:
return_series('aapl', 'mu')

AttributeError: 'str' object has no attribute 'Date'

In [10]:
def strat1(ticker):
    def get(ticker):
        temp_df = pd.read_csv("https://www.quandl.com/api/v3/datasets/YAHOO/"+ticker+".csv")
        temp_df.index = pd.to_datetime(temp_df.Date)
        temp_df = temp_df.sort()
        return temp_df
    ret_df = get(ticker)
    ret_df['ret'] = ret_df.Close.shift(-1) - ret_df.Close
    return_df = ret_df
    return_df['day1'] = np.where(return_df.ret > return_df.ret.shift(1), 1, 0)
    return_df['five_days'] = np.where(return_df.day1 + return_df.day1.shift(1) +
                                 return_df.day1.shift(2) + return_df.day1.shift(3) +
                                 return_df.day1.shift(4) + return_df.day1.shift(5) == 5, 1, 0)
    return_df['trade_yes'] = np.where(return_df.five_days == 1, 1, 0)
    return_df['ret_long'] = np.where(return_df.trade_yes == 1, return_df.ret + return_df.ret.shift(-1) +
                                 return_df.ret.shift(-2) + return_df.ret.shift(-3) +
                                 return_df.ret.shift(-4) + return_df.ret.shift(-5), 0)
    return_df['ret_short'] = -1 * return_df.ret_long
    return_df = return_df[return_df.index.year > 2000]
    length = return_df.ret_long.size / 15
    sharpe_df = return_df.ret_long.cumsum()[::length].shift(-1) - return_df.ret_long.cumsum()[::length]
    yearly_sharpe = sharpe_df.mean() / sharpe_df.std()
    return_df.ret_long.cumsum().plot(title = ticker+ "\n--Five Day Trade--\nSharpe = " + str(yearly_sharpe));
    return return_df